In [1]:
import pandas as pd

LINK DATA & POI

In [25]:
link_raw_data = pd.read_csv('link_seoul_ver5.csv', encoding='cp949')
poi = pd.read_csv('POI.csv', encoding='cp949')

Traffic accident Dataset

In [26]:
dataset = pd.DataFrame(columns=['datetime', 'lanes', 'limit', 'length', 
    'bump', 'camera', 'CONV', 'KINDER', 'TRAVEL','SCHOOL','BROKER', 'rain', 'temp', 'humid', 'visibility', 'dew_point', 'cloud', 'vaper_press', 'ground_temp', 'sun_diff_angle', 'sun_elevation'])

In [45]:
data = pd.read_csv('seoul_accident_drop.csv', encoding='cp949')
dataset['datetime'] = data['발생일시']

In [5]:
latitude = data['위도']
longitude = data['경도']

In [6]:
# 위도 경도 쌍으로 새로운 리스트 생성

lat_lon = []
for i in range(len(latitude)):
    lat_lon.append([longitude[i], latitude[i]])

link_mapping 함수 사용하여 위경도 정보를 링크 정보로 받아오기

In [7]:
import numpy as np
import haversine as hs 

By link_mapping.py

In [8]:
df = pd.read_csv('link_seoul_ver3.csv', encoding='cp949')
link = df['LINK_ID'].values
geometry = df['geometry']

geometry = geometry.str.replace('LINESTRING','')
geometry = geometry.str.replace('(','')
geometry = geometry.str.replace(')','')
geometry = geometry.str.replace(' ',',')
geometry = geometry.str.replace(',,',',')
geometry = geometry.str.split(',')
for i in range(len(geometry)):
    geometry[i].pop(0)

def distance(x1,y1,x2,y2,x3,y3):
    if x1 == x2:
        m = 0
    else:
        m = (y2-y1)/(x2-x1)
    a = m
    b = -1
    c = y1 - m*x1
    d = abs(a*x3 + b*y3 + c) / np.sqrt(a**2 + b**2)
    return d

def distance_link(link, geometry, x, y):
    d = []
    for i in range(0, len(geometry[link]) - 2, 2):
        x1 = float(geometry[link][i+1])
        y1 = float(geometry[link][i])
        x2 = float(geometry[link][i+3])
        y2 = float(geometry[link][i+2])
        d.append(distance(x1,y1,x2,y2,x,y))
    return min(d)

def distance_haversine(link, geometry, x, y):
    for i in range(len(geometry[link])):
        if i % 2 == 0:
            x1 = float(geometry[link][i+1])
            y1 = float(geometry[link][i])
            if hs.haversine((x,y),(x1,y1)) <= 0.3:
                return True
            else:
                continue
        else:
            continue
    

def link_haversine(geometry, x, y):
    link_haversine = []
    for i in range(len(geometry)):
        if distance_haversine(i,geometry,x,y) == True:
            link_haversine.append(i)
    return link_haversine

def link_shortest(link_haversine, geometry, x, y):
    d = []
    for i in range(len(link_haversine)):
        d.append(distance_link(link_haversine[i],geometry,x,y))
    return link_haversine[d.index(min(d))]


def mapping_link(x,y):
    linked_haversine = link_haversine(geometry, x, y)
    return link_shortest(linked_haversine, geometry, x, y)

C:\Users\정현진\AppData\Local\Temp\ipykernel_7136\3820450146.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geometry = geometry.str.replace('(','')
C:\Users\정현진\AppData\Local\Temp\ipykernel_7136\3820450146.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geometry = geometry.str.replace(')','')


In [4]:
link_id = []
lanes = []
limit = []
length = []
bump = []
camera = []

For Point of Interest

In [5]:
conv = []
kinder = []
travel = []
school = []
broker = []
# We will use CONv, KINDER, TRAVEL, SCHOOL, BROKER as POI

Static feature in Link Info

In [11]:
for i in range(len(lat_lon)):
    link_result = mapping_link(lat_lon[i][0], lat_lon[i][1])
    link_data = link_raw_data.loc[link_result]
    link_id.append(link_data['LINK_ID'])
    poi_data = poi.loc[link_result]
    lanes.append(link_data['LANES'])
    limit.append(link_data['MAX_SPD'])
    length.append(link_data['LENGTH'])
    camera.append(link_data['Speed_Camera'])
    bump.append(link_data['BUMP'])
    conv.append(poi_data['CONV'])
    kinder.append(poi_data['KINDER'])
    travel.append(poi_data['TRAVEL'])
    school.append(poi_data['SCHOOL'])
    broker.append(poi_data['BROKER'])
dataset['lanes'] = lanes
dataset['limit'] = limit
dataset['length'] = length
dataset['bump'] = bump
dataset['camera'] = camera
dataset['CONV'] = conv
dataset['KINDER'] = kinder
dataset['TRAVEL'] = travel
dataset['SCHOOL'] = school
dataset['BROKER'] = broker

Dynamic Feature in Accident incident Time Info

At First, by sun.py -> sun_elevation, sun_diff_angle

In [46]:
import math
import os.path
import numpy as np
import pandas as pd
from datetime import datetime

upper_path = os.getcwd()
if upper_path.split("\\")[-1] != "DataManager":
    upper_path = os.path.join(upper_path, "DataManager")

sun_csv = os.path.join(upper_path, "Datas", "link_seoul_ver4_utf8.csv")
link_df = pd.read_csv(sun_csv, encoding='utf8')

def calc_elevation(year=2020, month=1, day=1, hour=0, minute=0):
    '''
    주어진 시각의 태양 고도를 계산하는 함수입니다.
    기본적으로 서울의 위도와 경도를 사용합니다.
        Args:
            year `int`: 연도
            month `int`: 월
            day `int`: 일
            hour `int`: 시
            minute `int`: 분
        Return:
            elevation `float`: 태양 고도 (0° <= elevation <= 76.122°)
    '''
    d2r = math.pi / 180.0
    r2d = 180.0 / math.pi

    local_latitude = 37.378
    local_longitude = 127.148
    standard_longitude = 135

    day_of_year = datetime(year, month, day).timetuple().tm_yday

    B = (day_of_year - 1) * 360.0 / 365.0
    EOT = 229.2 * (0.000075
                   + 0.001868 * math.cos(B * d2r)
                   - 0.032077 * math.sin(B * d2r)
                   - 0.014615 * math.cos(2 * B * d2r)
                   - 0.040849 * math.sin(2 * B * d2r))

    # 시간각
    local_hour_decimal = hour + minute / 60.0
    delta_longitude = local_longitude - standard_longitude
    hour_angle = (local_hour_decimal * 60 + 4 * delta_longitude + EOT) / 60 * 15 - 180
    if hour_angle < -180:
        hour_angle += 360

    # 태양 적위
    solar_declination = 23.45 * math.sin((360.0 / 365.0 * (284 + day_of_year)) * d2r)

    # 태양 고도
    term_1 = math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination) * math.cos(d2r * hour_angle) \
             + math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination)
    solar_altitude = math.asin(term_1) * r2d

    return solar_altitude if solar_altitude > 0 else 0


def calc_angle(year=2020, month=1, day=1, hour=0, minute=0, road=0):
    '''
    주어진 시각의 태양 방위각과 주어진 도로와의 사잇각을 구하는 함수입니다.
    좌우 구분 없이 0° ~ 180°의 값을 가집니다.
        Args:
            year `int`: 연도
            month `int`: 월
            day `int`: 일
            hour `int`: 시
            minute `int`: 분
            road `int`: 도로 링크 아이디
        Return:
            azimuth `float`: 태양 방위각 (0° <= azimuth <= 180°)
    '''

    def calc_solar_azimuth():
        d2r = math.pi / 180.0
        r2d = 180.0 / math.pi

        local_latitude = 37.378
        local_longitude = 127.148
        standard_longitude = 135

        day_of_year = datetime(year, month, day).timetuple().tm_yday

        B = (day_of_year - 1) * 360.0 / 365.0
        EOT = 229.2 * (0.000075
                       + 0.001868 * math.cos(B * d2r)
                       - 0.032077 * math.sin(B * d2r)
                       - 0.014615 * math.cos(2 * B * d2r)
                       - 0.040849 * math.sin(2 * B * d2r))

        # 시간각
        local_hour_decimal = hour + minute / 60.0
        delta_longitude = local_longitude - standard_longitude
        hour_angle = (local_hour_decimal * 60 + 4 * delta_longitude + EOT) / 60 * 15 - 180
        if hour_angle < -180:
            hour_angle += 360

        # 태양 적위
        solar_declination = 23.45 * math.sin((360.0 / 365.0 * (284 + day_of_year)) * d2r)

        # 태양 고도
        term_1 = math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination) * math.cos(d2r * hour_angle) \
                 + math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination)
        solar_altitude = math.asin(term_1) * r2d

        # 태양 방위각
        term_2 = (math.sin(d2r * solar_altitude) * math.sin(d2r * local_latitude) - math.sin(d2r * solar_declination)) \
                 / (math.cos(d2r * solar_altitude) * math.cos(d2r * local_latitude))
        solar_azimuth = r2d * math.acos(term_2)
        solar_azimuth = solar_azimuth if hour_angle > 0 else 360 - solar_azimuth
        solar_azimuth = (180.0 + solar_azimuth) % 360.0
        return np.array([math.sin(solar_azimuth * d2r), math.cos(solar_azimuth * d2r)])

    def get_road_vector():
        df = link_df
        df = df[['LINK_ID', 'F_NODE', 'T_NODE']]
        df = df[df['LINK_ID'] == road]
        x0, y0 = map(float, (df['F_NODE'].values[0].split()))
        x1, y1 = map(float, (df['T_NODE'].values[0].split()))
        vec = np.array([x1 - x0, y1 - y0])
        # 정규화
        vec = vec / np.linalg.norm(vec)
        return vec

    solar_vec = calc_solar_azimuth()
    road_vec = get_road_vector()
    v = np.dot(solar_vec, road_vec) / (np.linalg.norm(solar_vec) * np.linalg.norm(road_vec))
    theta = np.arccos(v) * 180 / math.pi
    return theta

In [47]:
# data에서 발생일시에서 년, 월, 일, 시, 분을 각각 리스트로 추출

year = []
month = []
day = []
hour = []
minute = []
for i in range(len(data)):
    year.append(int(data['발생일시'][i][:4]))
    month.append(int(data['발생일시'][i][5:7]))
    day.append(int(data['발생일시'][i][8:10]))
    time = data['발생일시'][i][11:16]
    time = time.split(':')
    hour.append(int(time[0]))
    minute.append(int(time[1]))


In [ ]:
sun_ele = []
sun_angle = []

for i in range(len(data)):
    sun_ele.append(calc_elevation(year[i], month[i], day[i], hour[i], minute[i]))
    sun_angle.append(calc_angle(year[i], month[i], day[i], hour[i], minute[i], int(link_id[i])))

In [211]:
dataset['sun_elevation'] = sun_ele
dataset['sun_diff_angle'] = sun_angle

Weather feature attachment 


In [213]:
import datetime
weather = pd.read_csv('날씨/weather.csv', encoding='cp949')
weather = weather.set_index('date')
weather.index = pd.to_datetime(weather.index)
# year, month, day, hour, minute으로 특정 날짜의 날씨 데이터를 가져오기

# print(weather.loc['2020-01-01 8:00']['temp'])

In [214]:
rain = []
humid = []
visibility = []
dew_point = []
cloud = []
vaper_pressure = []
ground_temp = []
temp = []

In [215]:
for i in range(len(data)):
    rain.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['rain'])
    humid.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['humid'])
    visibility.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['visibility'])
    dew_point.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['dew_point'])
    cloud.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['cloud'])
    vaper_pressure.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['vaper_press'])
    ground_temp.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['ground_temp'])
    temp.append(weather.loc[f'{year[i]}-{month[i]}-{day[i]} {hour[i]}:00']['temp'])

In [216]:
dataset['rain'] = rain
dataset['humid'] = humid
dataset['visibility'] = visibility
dataset['dew_point'] = dew_point
dataset['cloud'] = cloud
dataset['vaper_press'] = vaper_pressure
dataset['ground_temp'] = ground_temp
dataset['temp'] = temp

In [218]:
dataset.to_csv('dataset.csv', index=False)

Not-Accident Dataset 

In [9]:
import pandas as pd

In [37]:
link = pd.read_csv('Not_Accident_dataset_link_list.csv', encoding="cp949")
link = list(link['link'])
link_raw = list(link_raw_data['LINK_ID'])

In [38]:
link_idx = []

In [39]:
for i in range(len(link)):
    link_id = link[i]
    for j in range(len(link)):
        if link_id == link_raw[j]:
            link_idx.append(j)
            break

In [40]:
lanes = []
limit = []
length = []
bump = []
camera = []
conv = []
kinder = []
travel = []
school = []
broker = []
# We will use CONv, KINDER, TRAVEL, SCHOOL, BROKER as POI

Static feature in link info(I will use previous dataset DataFrame)

In [42]:
for i in range(len(link)):
    link_result = link_idx[i]
    link_data = link_raw_data.iloc[link_result]
    poi_data = poi.loc[link_result]
    lanes.append(link_data['LANES'])
    limit.append(link_data['MAX_SPD'])
    length.append(link_data['LENGTH'])
    camera.append(link_data['Speed_Camera'])
    bump.append(link_data['BUMP'])
    conv.append(poi_data['CONV'])
    kinder.append(poi_data['KINDER'])
    travel.append(poi_data['TRAVEL'])
    school.append(poi_data['SCHOOL'])
    broker.append(poi_data['BROKER'])
dataset['lanes'] = lanes
dataset['limit'] = limit
dataset['length'] = length
dataset['bump'] = bump
dataset['camera'] = camera
dataset['CONV'] = conv
dataset['KINDER'] = kinder
dataset['TRAVEL'] = travel
dataset['SCHOOL'] = school
dataset['BROKER'] = broker

In [49]:
dataset.to_csv('dataset_non_accident.csv', index=False)